In [7]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET

from future.utils import iteritems

In [77]:
# type(iArray) :: print_debug
declare_block = """type({}) :: {}"""

### Generate the blocks for the dimensions

In [78]:
workfile = '/Users/pnorton/Projects/National_Hydrology_Model/src/fortran/prms6/src/xml/dimensions.xml'

xml_tree = xmlET.parse(workfile)
xml_root = xml_tree.getroot()

print(xml_root.tag)

dimensions


In [79]:
dimensions = {}
# Iterate over child nodes of root
for elem in xml_root.findall('dimension'):
    # print(elem.attrib.get('name'))
    name = elem.attrib.get('name')
    
    # Print the elements of the node
    # print(elem.find('desc').text)
    # print(elem.find('size').text)
    dim_size = int(elem.find('size').text)

    dimensions[name] = dim_size
    
dim_names = dimensions.keys()
dim_names.sort()

print(dimensions)

{'nhrucell': 0, 'one': 1, 'ndeplval': 0, 'nsnow': 0, 'nlakeelev': 0, 'nlapse': 0, 'ndays': 366, 'ngwcell': 0, 'npoigages': 0, 'ncascdgw': 0, 'ngw': 0, 'nsub': 0, 'nrain': 0, 'nssr': 0, 'nevap': 0, 'nmonths': 12, 'nhru': 0, 'nobs': 0, 'nexternal': 0, 'nlake': 0, 'nhumid': 0, 'ndepl': 0, 'nsegment': 0, 'nsol': 0, 'ntemp': 0, 'nconsumed': 0, 'ncascade': 0, 'nwateruse': 0, 'nratetbl': 0, 'nwind': 0}


In [80]:
# Open files to write the typedefs and code blocks to
basedir = '/Users/pnorton/Projects/National_Hydrology_Model/src/fortran/prms6'
typedef_hdl = open('{}/src/prmslib/classes/sm_control_typedef.inc'.format(basedir), 'w')
code_hdl = open('{}/src/prmslib/classes/sm_control_case_block.inc'.format(basedir), 'w')

tb = '  '

# Write the start of the select-case statement
code_hdl.write('select case(buf)\n')

code_footer = """
  case default
    ! Skip to the next ####
end select"""

### Generate the type definition code for dimensions

In [81]:
typedef_hdl.write('! Allowed dimensions\n')

for dim in dim_names:
    typedef_hdl.write('{}\n'.format(declare_block.format('iScalar', dim)))
#     print(declare_block.format('iScalar', dim))
    
typedef_hdl.write('\n')

In [82]:
# typedef_hdl.close()

### Code blocks for the select-case statement in the sm_control.f90

In [83]:
sc_code_block_array = """case('{0}')
    this%{0}%name = last
    call this%{0}%read(iUnit)
    line = line + this%{0}%size() + ENTRY_OFFSET"""

sc_code_block_scalar = """case('{0}')
    this%{0}%name = last
    call this%{0}%read(iUnit)
    line = line + 1 + ENTRY_OFFSET"""

### Generate the select case code for dimensions

In [84]:
code_hdl.write('  ! Dimensions\n')

for dim in dim_names:
    code_hdl.write('{}{}\n'.format(tb, sc_code_block_scalar.format(dim)))
#     print(sc_code_block_scalar.format(dim))

code_hdl.write('\n')

### Generate blocks for control parameters

In [85]:
workfile = '/Users/pnorton/Projects/National_Hydrology_Model/src/fortran/prms6/src/xml/control.xml'

xml_tree = xmlET.parse(workfile)
xml_root = xml_tree.getroot()

# print(xml_root.tag)

control = {}
# Iterate over child nodes of root
for elem in xml_root.findall('control_param'):
    # print(elem.attrib.get('name'))
    name = elem.attrib.get('name')
    
    # Print the elements of the node
    # print(elem.find('desc').text)
    # print(elem.find('size').text)
    ctl_type = int(elem.find('type').text)

    control[name] = ctl_type
    
ctl_names = control.keys()
ctl_names.sort()

# print(ctl_names)

### Generate the type definition code for control parameters

In [86]:
# Control file variables that should be arrays of values
array_ctl_vars = ['data_file', 'end_time', 'param_file', 'start_time', 'statVar_element', 
                  'statVar_names', 'aniOutVar_names', 'basinOutVar_names', 'mapOutVar_names',
                  'nhruOutVar_names', 'nsubOutVar_names', 'dispVar_element', 'dispVar_names',
                  'dispVar_plot']

datatype_map_arrays = {1: 'iArray', 2: 'rArray', 3: 'rArray', 4:'sArray'}

# NOTE: Type 4 is remains an array instead of a scalar for now
datatype_map_scalars = {1: 'iScalar', 2: 'rScalar', 3: 'rScalar', 4: 'sArray'}

typedef_hdl.write('! Control variables\n')

for ctl in ctl_names:
    if ctl in array_ctl_vars:
        typedef_hdl.write('{}\n'.format(code_block.format(datatype_map_arrays[control[ctl]], ctl)))
    else:
        typedef_hdl.write('{}\n'.format(code_block.format(datatype_map_scalars[control[ctl]], ctl)))
        
typedef_hdl.write('\n')
typedef_hdl.close()

### Generate select case code for control parameters

In [87]:
code_hdl.write('  ! Control variables\n')

for ctl in ctl_names:
    if ctl in array_ctl_vars:
        code_hdl.write('{}{}\n'.format(tb, sc_code_block_array.format(ctl)))
    else:
        code_hdl.write('{}{}\n'.format(tb, sc_code_block_scalar.format(ctl)))
        
code_hdl.write(code_footer)
code_hdl.write('\n')
code_hdl.close()